# 🤖 Prédictions Ligue 1 - Approche Machine Learning

Ce notebook utilise des techniques de **Machine Learning** pour prédire les résultats de matchs de Ligue 1.

## 🎯 Différences avec l'approche simple

### Approche Simple (`predictions_ligue1.ipynb`)
- Score composite manuel : `forme + (buts_marqués - buts_encaissés) × 0.5`
- Seuil de décision fixe
- Pas d'apprentissage sur données historiques

### Approche Machine Learning (ce notebook)
- **Modèles supervisés** : Random Forest, Régression Logistique
- **Features multiples** : domicile/extérieur, forme, statistiques offensives/défensives, head-to-head
- **Apprentissage** sur données historiques de matchs
- **Probabilités** de victoire/nul/défaite
- **Validation** avec métriques (accuracy, précision, rappel)

## 📋 Prérequis

1. Clé API-Football configurée dans `.env`
2. Dépendances installées : `scikit-learn`, `pandas`, `numpy`

In [1]:
# Installation des dépendances (si nécessaire)
# !pip install requests pandas numpy scikit-learn python-dotenv matplotlib seaborn

import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime
import os
from dotenv import load_dotenv

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ Bibliothèques importées")

✅ Bibliothèques importées


In [2]:
# Configuration - Chargement depuis .env
load_dotenv()

API_KEY = os.getenv("API_FOOTBALL_KEY", "votre_clé_api_ici")
LIGUE1_ID = int(os.getenv("LIGUE1_ID", 61))
CURRENT_SEASON = int(os.getenv("CURRENT_SEASON", 2025))

print(f"✅ Configuration chargée : Ligue {LIGUE1_ID}, Saison {CURRENT_SEASON}")
print(f"📅 Saison {CURRENT_SEASON}-{CURRENT_SEASON+1}")

if API_KEY == "votre_clé_api_ici":
    print("⚠️ N'oubliez pas de configurer votre API_FOOTBALL_KEY dans le fichier .env")

✅ Configuration chargée : Ligue 61, Saison 2025
📅 Saison 2025-2026


## 🔧 Classe FootballAPI (réutilisée)

Utilise le même système de cache pour économiser les appels API.

In [3]:
import time
import hashlib

class FootballAPI:
    """Classe pour interagir avec l'API-Football avec système de cache"""
    
    def __init__(self, api_key, league_id=61, season=2025, cache_duration=3600, cache_dir='cache'):
        self.base_url = "https://v3.football.api-sports.io"
        self.headers = {'x-apisports-key': api_key}
        self.league_id = league_id
        self.season = season
        self.cache_duration = cache_duration
        self.cache_dir = cache_dir
        self.api_calls = 0
        
        if not os.path.exists(self.cache_dir):
            os.makedirs(self.cache_dir)
    
    def _get_cache_filename(self, cache_key):
        hash_key = hashlib.md5(cache_key.encode()).hexdigest()
        return os.path.join(self.cache_dir, f"{hash_key}.json")
    
    def _get_from_cache(self, cache_key):
        cache_file = self._get_cache_filename(cache_key)
        if os.path.exists(cache_file):
            file_age = time.time() - os.path.getmtime(cache_file)
            if file_age < self.cache_duration:
                with open(cache_file, 'r', encoding='utf-8') as f:
                    return json.load(f)
        return None
    
    def _save_to_cache(self, cache_key, data):
        cache_file = self._get_cache_filename(cache_key)
        with open(cache_file, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
    
    def _make_request(self, endpoint, params):
        cache_key = f"{endpoint}?" + '&'.join([f"{k}={v}" for k, v in sorted(params.items())])
        
        cached_data = self._get_from_cache(cache_key)
        if cached_data:
            return cached_data
        
        url = f"{self.base_url}{endpoint}"
        response = requests.get(url, headers=self.headers, params=params)
        self.api_calls += 1
        
        if response.status_code == 200:
            data = response.json()
            self._save_to_cache(cache_key, data)
            return data
        return None
    
    def get_fixtures(self, season=None, status='FT'):
        """Récupère les matchs terminés d'une saison"""
        params = {
            'league': self.league_id,
            'season': season or self.season,
            'status': status
        }
        return self._make_request('/fixtures', params)
    
    def get_team_statistics(self, team_id, season=None):
        """Récupère les statistiques d'une équipe"""
        params = {
            'league': self.league_id,
            'season': season or self.season,
            'team': team_id
        }
        return self._make_request('/teams/statistics', params)
    
    def get_head_to_head(self, team1_id, team2_id, last=10):
        """Récupère l'historique des confrontations"""
        params = {'h2h': f"{team1_id}-{team2_id}", 'last': last}
        return self._make_request('/fixtures/headtohead', params)

# Initialisation de l'API
api = FootballAPI(API_KEY, LIGUE1_ID, CURRENT_SEASON, cache_duration=7200)  # Cache 2h
print("✅ API initialisée")

✅ API initialisée


## 📊 Collecte des Données Historiques

Pour entraîner un modèle ML, nous avons besoin de données historiques de matchs terminés.

In [4]:
def collect_match_data(season):
    """Collecte les données de matchs terminés pour une saison"""
    print(f"🔄 Récupération des matchs de la saison {season}...")
    
    fixtures = api.get_fixtures(season=season, status='FT')
    
    if not fixtures or not fixtures.get('response'):
        print(f"❌ Aucune donnée disponible pour la saison {season}")
        return pd.DataFrame()
    
    matches = []
    for match in fixtures['response']:
        home_team = match['teams']['home']
        away_team = match['teams']['away']
        goals = match['goals']
        
        # Déterminer le résultat (1=victoire domicile, 0=nul, 2=victoire extérieur)
        if goals['home'] > goals['away']:
            result = 1  # Victoire domicile
        elif goals['home'] < goals['away']:
            result = 2  # Victoire extérieur
        else:
            result = 0  # Nul
        
        matches.append({
            'match_id': match['fixture']['id'],
            'date': match['fixture']['date'],
            'home_team_id': home_team['id'],
            'home_team_name': home_team['name'],
            'away_team_id': away_team['id'],
            'away_team_name': away_team['name'],
            'home_goals': goals['home'],
            'away_goals': goals['away'],
            'result': result,
            'season': season
        })
    
    df = pd.DataFrame(matches)
    print(f"✅ {len(df)} matchs collectés pour la saison {season}")
    return df

# Collecter les données de plusieurs saisons pour l'entraînement
# Note: Adaptez les saisons selon la disponibilité des données
seasons_to_collect = [2024, 2023, 2022]  # 3 dernières saisons complètes

all_matches = []
for season in seasons_to_collect:
    df_season = collect_match_data(season)
    if not df_season.empty:
        all_matches.append(df_season)
    time.sleep(1)  # Petit délai entre les requêtes

if all_matches:
    df_matches = pd.concat(all_matches, ignore_index=True)
    print(f"\n📊 Total: {len(df_matches)} matchs sur {len(seasons_to_collect)} saisons")
    display(df_matches.head(10))
else:
    print("⚠️ Aucune donnée collectée. Vérifiez votre connexion API.")
    df_matches = pd.DataFrame()

🔄 Récupération des matchs de la saison 2024...
✅ 307 matchs collectés pour la saison 2024
🔄 Récupération des matchs de la saison 2023...
✅ 307 matchs collectés pour la saison 2023
🔄 Récupération des matchs de la saison 2022...
✅ 380 matchs collectés pour la saison 2022

📊 Total: 994 matchs sur 3 saisons


,match_id,date,home_team_id,home_team_name,away_team_id,away_team_name,home_goals,away_goals,result,season
0,1213754,2024-08-16T18:45:00+00:00,111,Le Havre,85,Paris Saint Germain,1,4,2,2024
1,1213747,2024-08-17T15:00:00+00:00,106,Stade Brestois 29,81,Marseille,1,5,2,2024
2,1213752,2024-08-17T17:00:00+00:00,93,Reims,79,Lille,0,2,2,2024
3,1213751,2024-08-17T19:00:00+00:00,91,Monaco,1063,Saint Etienne,1,0,1,2024
4,1213753,2024-08-18T13:00:00+00:00,108,Auxerre,84,Nice,2,1,1,2024
5,1213750,2024-08-18T15:00:00+00:00,77,Angers,116,Lens,0,1,2,2024
6,1213748,2024-08-18T15:00:00+00:00,82,Montpellier,95,Strasbourg,1,1,0,2024
7,1213749,2024-08-18T15:00:00+00:00,96,Toulouse,83,Nantes,0,0,0,2024
8,1213755,2024-08-18T18:45:00+00:00,94,Rennes,80,Lyon,3,0,1,2024
9,1213756,2024-08-23T18:45:00+00:00,85,Paris Saint Germain,82,Montpellier,6,0,1,2024


## 🔨 Feature Engineering

Création de variables (features) pour le modèle ML à partir des statistiques d'équipes.

In [5]:
def calculate_form_score(form_string):
    """Calcule un score de forme (W=3, D=1, L=0)"""
    if not form_string:
        return 0
    return sum(3 if r == 'W' else 1 if r == 'D' else 0 for r in form_string)

def get_team_features(team_id, season, is_home=True):
    """Récupère les features d'une équipe pour une saison donnée"""
    stats = api.get_team_statistics(team_id, season)
    
    if not stats or not stats.get('response'):
        return None
    
    s = stats['response']
    fixtures = s['fixtures']
    goals = s['goals']
    
    # Sélectionner les stats domicile ou extérieur
    location = 'home' if is_home else 'away'
    
    features = {
        'played': fixtures['played'].get(location, 0),
        'wins': fixtures['wins'].get(location, 0),
        'draws': fixtures['draws'].get(location, 0),
        'loses': fixtures['loses'].get(location, 0),
        'goals_for': goals['for']['total'].get(location, 0) or 0,
        'goals_against': goals['against']['total'].get(location, 0) or 0,
        'form_score': calculate_form_score(s.get('form'))
    }
    
    # Calculer les moyennes
    if features['played'] > 0:
        features['avg_goals_for'] = features['goals_for'] / features['played']
        features['avg_goals_against'] = features['goals_against'] / features['played']
        features['win_rate'] = features['wins'] / features['played']
    else:
        features['avg_goals_for'] = 0
        features['avg_goals_against'] = 0
        features['win_rate'] = 0
    
    features['goal_diff'] = features['goals_for'] - features['goals_against']
    
    return features

def create_match_features(home_team_id, away_team_id, season):
    """Crée les features pour un match donné"""
    
    # Features équipe domicile
    home_features = get_team_features(home_team_id, season, is_home=True)
    # Features équipe extérieur
    away_features = get_team_features(away_team_id, season, is_home=False)
    
    if not home_features or not away_features:
        return None
    
    # Combiner les features avec préfixes
    features = {}
    for key, value in home_features.items():
        features[f'home_{key}'] = value
    for key, value in away_features.items():
        features[f'away_{key}'] = value
    
    # Features comparatives
    features['goal_diff_difference'] = home_features['goal_diff'] - away_features['goal_diff']
    features['form_difference'] = home_features['form_score'] - away_features['form_score']
    features['win_rate_difference'] = home_features['win_rate'] - away_features['win_rate']
    
    return features

print("✅ Fonctions de feature engineering définies")

✅ Fonctions de feature engineering définies


In [6]:
# Créer les features pour tous les matchs collectés
if not df_matches.empty:
    print("🔨 Création des features pour chaque match...")
    
    features_list = []
    labels = []
    
    # Limiter au premier 50 matchs pour économiser les appels API
    sample_size = min(50, len(df_matches))
    print(f"📊 Traitement de {sample_size} matchs (échantillon)")
    
    for idx, row in df_matches.head(sample_size).iterrows():
        features = create_match_features(
            row['home_team_id'],
            row['away_team_id'],
            row['season']
        )
        
        if features:
            features_list.append(features)
            labels.append(row['result'])
        
        # Afficher progression tous les 10 matchs
        if (idx + 1) % 10 == 0:
            print(f"  Traité: {idx + 1}/{sample_size} matchs")
        
        time.sleep(0.5)  # Délai entre requêtes
    
    # Créer le DataFrame de features
    df_features = pd.DataFrame(features_list)
    df_features['result'] = labels
    
    print(f"\n✅ Dataset créé: {len(df_features)} matchs avec {len(df_features.columns)-1} features")
    print(f"\n📊 Distribution des résultats:")
    print(df_features['result'].value_counts().sort_index())
    print("\n  0 = Nul")
    print("  1 = Victoire domicile")
    print("  2 = Victoire extérieur")
    
    display(df_features.head())
else:
    print("⚠️ Pas de données de matchs disponibles")

🔨 Création des features pour chaque match...
📊 Traitement de 50 matchs (échantillon)
  Traité: 10/50 matchs
  Traité: 20/50 matchs
  Traité: 30/50 matchs
  Traité: 40/50 matchs
  Traité: 50/50 matchs

✅ Dataset créé: 9 matchs avec 25 features

📊 Distribution des résultats:
result
0    2
1    2
2    5
Name: count, dtype: int64

  0 = Nul
  1 = Victoire domicile
  2 = Victoire extérieur


,home_played,home_wins,home_draws,home_loses,home_goals_for,home_goals_against,home_form_score,home_avg_goals_for,home_avg_goals_against,home_win_rate,...,away_goals_against,away_form_score,away_avg_goals_for,away_avg_goals_against,away_win_rate,away_goal_diff,goal_diff_difference,form_difference,win_rate_difference,result
0,17,3,2,12,15,41,34,0.882353,2.411765,0.176471,...,19,84,2.764706,1.117647,0.705882,28,-54,-50,-0.529412,2
1,17,10,3,4,31,21,50,1.823529,1.235294,0.588235,...,24,65,1.941176,1.411765,0.588235,9,1,-15,0.000000,2
2,18,4,4,10,18,28,34,1.000000,1.555556,0.222222,...,18,60,1.235294,1.058824,0.352941,3,-13,-26,-0.130719,2
3,17,10,4,3,41,23,65,2.411765,1.352941,0.588235,...,23,34,0.944444,1.277778,0.222222,-6,24,31,0.366013,0
4,17,11,4,2,31,18,60,1.823529,1.058824,0.647059,...,19,84,2.764706,1.117647,0.705882,28,-15,-24,-0.058824,2


## 🎓 Entraînement du Modèle

Utilisation de Random Forest pour classifier les résultats de matchs.

In [7]:
if 'df_features' in locals() and not df_features.empty:
    # Séparer features (X) et labels (y)
    X = df_features.drop('result', axis=1)
    y = df_features['result']
    
    # Split train/test (80/20)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"📊 Données d'entraînement: {len(X_train)} matchs")
    print(f"📊 Données de test: {len(X_test)} matchs")
    
    # Normalisation des features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Entraînement Random Forest
    print("\n🌲 Entraînement du Random Forest...")
    rf_model = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    )
    rf_model.fit(X_train_scaled, y_train)
    
    # Prédictions
    y_pred = rf_model.predict(X_test_scaled)
    
    # Évaluation
    accuracy = accuracy_score(y_test, y_pred)
    print(f"\n✅ Modèle entraîné!")
    print(f"🎯 Accuracy: {accuracy:.2%}")
    
    print("\n📊 Rapport de classification:")
    print(classification_report(y_test, y_pred, target_names=['Nul', 'Victoire Dom.', 'Victoire Ext.']))
    
    # Matrice de confusion
    print("\n🔢 Matrice de confusion:")
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    
    # Importance des features
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': rf_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\n⭐ Top 10 features les plus importantes:")
    display(feature_importance.head(10))
    
else:
    print("⚠️ Pas de données pour entraîner le modèle")

ValueError: The test_size = 2 should be greater or equal to the number of classes = 3

## 🔮 Fonction de Prédiction

Utiliser le modèle entraîné pour prédire le résultat d'un match.

In [ ]:
def predict_match_ml(home_team_id, away_team_id, home_team_name=None, away_team_name=None):
    """Prédiction ML d'un match avec probabilités"""
    
    if 'rf_model' not in locals() and 'rf_model' not in globals():
        return "❌ Modèle non entraîné. Exécutez d'abord les cellules d'entraînement."
    
    # Créer les features pour ce match
    features = create_match_features(home_team_id, away_team_id, CURRENT_SEASON)
    
    if not features:
        return "❌ Impossible de récupérer les statistiques des équipes"
    
    # Préparer les features dans le bon ordre
    X_new = pd.DataFrame([features])[X.columns]
    X_new_scaled = scaler.transform(X_new)
    
    # Prédiction
    prediction = rf_model.predict(X_new_scaled)[0]
    probabilities = rf_model.predict_proba(X_new_scaled)[0]
    
    # Affichage
    home_name = home_team_name or f"Équipe {home_team_id}"
    away_name = away_team_name or f"Équipe {away_team_id}"
    
    print(f"⚔️  {home_name} vs {away_name}\n")
    print(f"🤖 Prédiction Machine Learning:\n")
    
    # Probabilités
    print(f"📊 Probabilités:")
    print(f"   🤝 Match nul: {probabilities[0]:.1%}")
    print(f"   🏠 Victoire {home_name}: {probabilities[1]:.1%}")
    print(f"   ✈️  Victoire {away_name}: {probabilities[2]:.1%}")
    
    # Résultat final
    print(f"\n🎯 Prédiction finale:")
    if prediction == 0:
        result = f"🤝 Match nul probable"
    elif prediction == 1:
        result = f"🏆 Victoire probable de {home_name}"
    else:
        result = f"🏆 Victoire probable de {away_name}"
    
    print(f"   {result}")
    print(f"   Confiance: {max(probabilities):.1%}")
    
    return result

print("✅ Fonction de prédiction ML définie")

## 🎮 Exemples de Prédictions

Tester le modèle sur des matchs connus.

In [ ]:
# Exemple 1: PSG vs Marseille
if 'rf_model' in locals():
    predict_match_ml(85, 81, "Paris Saint Germain", "Olympique de Marseille")
else:
    print("⚠️ Entraînez d'abord le modèle avant de faire des prédictions")

In [ ]:
# Exemple 2: Monaco vs Lyon
if 'rf_model' in locals():
    predict_match_ml(91, 80, "AS Monaco", "Olympique Lyonnais")
else:
    print("⚠️ Entraînez d'abord le modèle avant de faire des prédictions")

In [ ]:
# Exemple 3: Lille vs Nice
if 'rf_model' in locals():
    predict_match_ml(79, 33, "LOSC Lille", "OGC Nice")
else:
    print("⚠️ Entraînez d'abord le modèle avant de faire des prédictions")

## 📈 Visualisation des Performances

Graphiques pour analyser les performances du modèle.

In [ ]:
if 'rf_model' in locals() and 'y_test' in locals():
    # Matrice de confusion visualisée
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Nul', 'Victoire Dom.', 'Victoire Ext.'],
                yticklabels=['Nul', 'Victoire Dom.', 'Victoire Ext.'])
    plt.title('Matrice de Confusion - Random Forest')
    plt.ylabel('Valeur Réelle')
    plt.xlabel('Prédiction')
    plt.show()
    
    # Importance des features
    plt.figure(figsize=(10, 8))
    top_features = feature_importance.head(15)
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Importance')
    plt.title('Top 15 Features par Importance')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Modèle non entraîné")

## 📊 Comparaison avec l'Approche Simple

Tableau comparatif des deux approches.

In [ ]:
comparison = pd.DataFrame({
    'Critère': [
        'Complexité',
        'Données historiques',
        'Features utilisées',
        'Sortie',
        'Validation',
        'Domicile/Extérieur',
        'Appels API',
        'Précision estimée'
    ],
    'Approche Simple': [
        'Faible',
        'Non',
        '3 (forme, buts pour, buts contre)',
        'Score composite + seuil',
        'Aucune',
        'Non pris en compte',
        'Minimal (2 par match)',
        '~55-60%'
    ],
    'Approche ML': [
        'Élevée',
        'Oui (3 saisons)',
        '20+ (stats domicile/extérieur séparées)',
        'Probabilités + classe prédite',
        'Train/test split + métriques',
        'Oui (features distinctes)',
        'Important (entraînement initial)',
        '~60-70% (dépend des données)'
    ]
})

print("📊 Comparaison des Approches de Prédiction\n")
display(comparison)

## 🚀 Prochaines Améliorations

### Modèles plus avancés
- 🧠 **Deep Learning** : Réseaux de neurones avec TensorFlow/PyTorch
- 🌳 **Gradient Boosting** : XGBoost, LightGBM pour meilleure performance
- 📊 **Ensemble methods** : Combiner plusieurs modèles

### Features supplémentaires
- 📈 **xG (Expected Goals)** : Qualité des occasions créées
- 👥 **Head-to-head** : Historique des confrontations directes
- 🏥 **Blessures/Suspensions** : État des effectifs
- ⏱️ **Temps de repos** : Jours entre matchs
- 🌦️ **Météo** : Conditions climatiques
- 💰 **Cotes bookmakers** : Signal fort d'informations

### Optimisation du modèle
- 🔍 **Grid Search** : Optimisation des hyperparamètres
- ⚖️ **Gestion du déséquilibre** : SMOTE pour classes minoritaires
- 🎲 **Séries temporelles** : Prendre en compte l'ordre chronologique
- 📉 **Feature selection** : Réduire les features redondantes

### Validation
- 🔄 **Cross-validation** : Sur vraies séquences temporelles
- 📅 **Backtest** : Tester sur saison complète avant déploiement
- 📊 **Calibration** : Ajuster les probabilités prédites

## 💾 Sauvegarde du Modèle

In [ ]:
# Sauvegarder le modèle et le scaler pour utilisation ultérieure
if 'rf_model' in locals():
    import pickle
    
    # Créer dossier models s'il n'existe pas
    if not os.path.exists('models'):
        os.makedirs('models')
    
    # Sauvegarder
    with open('models/random_forest_model.pkl', 'wb') as f:
        pickle.dump(rf_model, f)
    
    with open('models/scaler.pkl', 'wb') as f:
        pickle.dump(scaler, f)
    
    with open('models/feature_columns.pkl', 'wb') as f:
        pickle.dump(X.columns.tolist(), f)
    
    print("💾 Modèle sauvegardé dans le dossier 'models/'")
    print("   - random_forest_model.pkl")
    print("   - scaler.pkl")
    print("   - feature_columns.pkl")
else:
    print("⚠️ Aucun modèle à sauvegarder")

## 📚 Ressources et Documentation

- 📖 [Documentation complète de la méthode simple](docs/prediction-simple.md)
- 🔗 [API-Football Documentation](https://www.api-football.com/documentation-v3)
- 🤖 [Scikit-learn Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

## 🎯 IDs d'équipes Ligue 1

- PSG: 85
- Marseille: 81
- Monaco: 91
- Lyon: 80
- Lille: 79
- Rennes: 94
- Nice: 33
- Lens: 116